# UNData API Exercise
In this exercise, you'll redo the data gathering phase of the UNData Exploration project by using APIs instead of downloading csv files.

You'll make use of the World Bank Indicators API. Note that this API does not require an API key. Before attempting the exercise, it would be a good idea to skim through the Documentation page and to check out the Basic Call Structure article.

In [1]:
import pandas as pd
import requests

In [2]:
# CONSTANTS
base_url = "https://api.worldbank.org/V2/"
indicator_endpoint = "country/all/indicator/"
params = {
    "format": "json",
    "Per_page": 20000
}

## 1. Use the API to get all available data for the GDP per capita, PPP (constant 2021 international $) indicator.
Hint: this indicator has code "NY.GDP.PCAP.PP.KD". Adjust the query parameters so that you can retrieve all available rows. Convert the results to a DataFrame.

In [3]:
# GDP per capita, PPP (constant 2021 international $) indicator
indicator = "NY.GDP.PCAP.PP.KD"

# SET URL
url = base_url + indicator_endpoint + indicator

# MAKE REQUEST
response = requests.get(url=url, params=params)

# CONVERT DATA TO JSON
data = response.json()

# CONVERT JSON DATA TO A DATAFRAME WITH NORMALIZED DATA
gdp_df = pd.json_normalize(data=data[1])
gdp_df.head(2)

,countryiso3code,date,value,unit,obs_status,decimal,indicator.id,indicator.value,country.id,country.value
0,AFE,2024,3968.963751,,,0,NY.GDP.PCAP.PP.KD,"GDP per capita, PPP (constant 2021 internation...",ZH,Africa Eastern and Southern
1,AFE,2023,3948.142721,,,0,NY.GDP.PCAP.PP.KD,"GDP per capita, PPP (constant 2021 internation...",ZH,Africa Eastern and Southern


## 2. Now, use the API to get all available data for Life expectancy at birth, total (years).
This indicator has code "SP.DYN.LE00.IN". Again, convert the results to a DataFrame.

In [4]:
# Life expectancy at birth, total (years) indicator
indicator = "SP.DYN.LE00.IN"

# SET URL
url = base_url + indicator_endpoint + indicator

# MAKE REQUEST
response = requests.get(url=url, params=params)

# CONVERT DATA TO JSON
data = response.json()

# CONVERT JSON DATA TO A DATAFRAME WITH NORMALIZED DATA
le_df = pd.json_normalize(data=data[1])
le_df.head(2)

,countryiso3code,date,value,unit,obs_status,decimal,indicator.id,indicator.value,country.id,country.value
0,AFE,2024,NaN,,,0,SP.DYN.LE00.IN,"Life expectancy at birth, total (years)",ZH,Africa Eastern and Southern
1,AFE,2023,65.146291,,,0,SP.DYN.LE00.IN,"Life expectancy at birth, total (years)",ZH,Africa Eastern and Southern


## 3. Merge the two results DataFrames together.
You may want to rename or drop columns prior to merging.

In [5]:
# CLEAN COLUMNS
gdp_df = (
    gdp_df
    .drop(columns=['unit', 'obs_status', 'decimal', 'indicator.id', 'indicator.value'])
    .rename(columns={
        "countryiso3code": "ISO3",
        "value": "GDP_per_capita",
        "country.id": "ISO2",
        "country.value": "country"
    })
)
gdp_df.head(2)

,ISO3,date,GDP_per_capita,ISO2,country
0,AFE,2024,3968.963751,ZH,Africa Eastern and Southern
1,AFE,2023,3948.142721,ZH,Africa Eastern and Southern


In [6]:
# CLEAN COLUMNS
le_df = (
    le_df
    .drop(columns=['unit', 'obs_status', 'decimal', 'indicator.id', 'indicator.value'])
    .rename(columns={
        "countryiso3code": "ISO3",
        "value": "life_expectancy",
        "country.id": "ISO2",
        "country.value": "country"
    })
)
le_df.head(2)

,ISO3,date,life_expectancy,ISO2,country
0,AFE,2024,NaN,ZH,Africa Eastern and Southern
1,AFE,2023,65.146291,ZH,Africa Eastern and Southern


In [7]:
# MERGE GDP WITH LE
gdp_le_df = pd.merge(
    left=gdp_df,
    right=le_df,
    how="inner",
    validate="1:1"
)
gdp_le_df.head(2)

,ISO3,date,GDP_per_capita,ISO2,country,life_expectancy
0,AFE,2024,3968.963751,ZH,Africa Eastern and Southern,NaN
1,AFE,2023,3948.142721,ZH,Africa Eastern and Southern,65.146291


## 4. You can also get more information about the available countries (region, capital city, income level classification, etc.) by using the Country API.
Use this API to pull in all available data. Merge this with your other datasets. Use this to now remove the rows that correspond to regions and not countries.

In [8]:
# CONSTANTS
country_endpoint = "country/"

# SET URL
url = base_url + country_endpoint

# MAKE REQUEST
response = requests.get(url=url, params=params)

# CONVERT DATA TO JSON
data = response.json()

# CONVERT JSON DATA TO A DATAFRAME WITH NORMALIZED DATA
country_df = pd.json_normalize(data=data[1])
country_df.head(2)

,id,iso2Code,name,capitalCity,longitude,latitude,region.id,region.iso2code,region.value,adminregion.id,adminregion.iso2code,adminregion.value,incomeLevel.id,incomeLevel.iso2code,incomeLevel.value,lendingType.id,lendingType.iso2code,lendingType.value
0,ABW,AW,Aruba,Oranjestad,-70.0167,12.5167,LCN,ZJ,Latin America & Caribbean,,,,HIC,XD,High income,LNX,XX,Not classified
1,AFE,ZH,Africa Eastern and Southern,,,,NA,NA,Aggregates,,,,NA,NA,Aggregates,,,Aggregates


In [9]:
# CLEAN COLUMNS
country_df = country_df.rename(columns={
    "id": "ISO3",
    "iso2Code": "ISO2",
    "name": "country"
})
country_df.head(2)

,ISO3,ISO2,country,capitalCity,longitude,latitude,region.id,region.iso2code,region.value,adminregion.id,adminregion.iso2code,adminregion.value,incomeLevel.id,incomeLevel.iso2code,incomeLevel.value,lendingType.id,lendingType.iso2code,lendingType.value
0,ABW,AW,Aruba,Oranjestad,-70.0167,12.5167,LCN,ZJ,Latin America & Caribbean,,,,HIC,XD,High income,LNX,XX,Not classified
1,AFE,ZH,Africa Eastern and Southern,,,,NA,NA,Aggregates,,,,NA,NA,Aggregates,,,Aggregates


In [10]:
# MERGE GDP WITH LE
gdp_le_country_df = pd.merge(
    left=gdp_le_df,
    right=country_df,
    how="inner",
    on=["ISO3", "ISO2", "country"],
    validate="m:1"
)
gdp_le_country_df.head(2)

,ISO3,date,GDP_per_capita,ISO2,country,life_expectancy,capitalCity,longitude,latitude,region.id,...,region.value,adminregion.id,adminregion.iso2code,adminregion.value,incomeLevel.id,incomeLevel.iso2code,incomeLevel.value,lendingType.id,lendingType.iso2code,lendingType.value
0,AFE,2024,3968.963751,ZH,Africa Eastern and Southern,NaN,,,,NA,...,Aggregates,,,,NA,NA,Aggregates,,,Aggregates
1,AFE,2023,3948.142721,ZH,Africa Eastern and Southern,65.146291,,,,NA,...,Aggregates,,,,NA,NA,Aggregates,,,Aggregates


In [11]:
# REMOVE ROWS CORRESPONDING TO REGIONS
drop_index_list = (
    gdp_le_country_df
    [(gdp_le_country_df['region.value'] == "Aggregates") | (gdp_le_country_df['ISO3'] == "")]
    .index
)

gdp_le_country_df = gdp_le_country_df.drop(index=drop_index_list)
len(gdp_le_country_df['country'].unique())

217

# Bonus Questions: After doing a basic request to get all records, you can attempt these exercises to get additional practice using query parameters.

## 1. Adjust your request so that it returns data just for the United States.

## 2. Adjust your request so that it returns data just for the United States for the year 2021.

## 3. Adjust your request so that it returns data just for the United States for the years 2000 through 2021.

## 4. Adjust your request so that it returns data for the United States and Canada for the years 2000 through 2021.

## 5. If you haven't already done so and you would like to get some additional practice using loops, use the page parameter in order to pull all records. Do not change the value of the per_page parameter. You will likely need to utilize a loop of some kind in order to pull all records.

## 6. The endpoint "http://api.worldbank.org/v2/indicator" will return information about all available indicators. See Indicator API Queries. Use this endpoint to find the code for the "Public Expenditure on Education (% GDP)" indicator.